In [1]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

# word_to_ix 는 사전의 각 단어를 고유한 정수로 매핑하고
# 그것은 BoW 벡터에서 자신의 색인이 됩니다.
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [3]:
class BoWClassifier(nn.Module):  # nn.Module로 부터 상속 받기 !

    def __init__(self, num_labels, vocab_size):
        # calls the init function of nn.Module의 초기화 함수 호출.  Dont get confused by syntax,
        # 문법에 혼란스러워 하지 마시고 단지 항상 nn.Module 에서 수행하십시오.
        super(BoWClassifier, self).__init__()

        # 필요한 파라미터를 정의 하십시오. 이 경우에는 아핀 매핑의 매개 변수 인 A와 b가 필요합니다.
        # Torch는 아핀 맵을 제공하는 nn.Linear()를 정의합니다
        # 입력 차원이 vocab_size이고 출력이 num_labels 인 이유를 이해했는지 확인하십시오!
        self.linear = nn.Linear(vocab_size, num_labels)

        # 주의! 비선형성 Log Softmax에는 파라미터가 없습니다!
        # 그래서 여기에 대해 걱정할 필요가 없습니다.

    def forward(self, bow_vec):
        # 선형 계층를 통해 입력을 전달한 다음 log_softmax로 전달합니다.
        # 많은 비선형성 및 기타 기능이 torch.nn.functional 에 있습니다

        return F.log_softmax(self.linear(bow_vec), dim=1)


In [4]:
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)


def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [6]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)


# 모델은 자신의 파라미터를 알고 있습니다. 아래에 있는 첫번째 출력은 A 두번째는 b 입니다.
# 모듈의 __init__ 함수에서 클래스 변수에 구성 요소를 할당 할 때마다 다음 행을 사용하여 완료합니다.
# self.linear = nn.Linear(...)
# 그런 다음 PyTorch 개발자의 Python 마법을 통해, 모듈(이 경우 BoWClassifier)은
# nn.Linear 파라미터에 대한 지식을 저장합니다
for param in model.parameters():
    print(param)

# 모델을 실행하려면 BoW 벡터를 전달합니다.
# 여기서 우리는 학습 할 필요가 없으므로 코드는 torch.no_grad()로 싸여 있습니다.
with torch.no_grad():
    sample = data[0]
    bow_vector = make_bow_vector(sample[0], word_to_ix)
    log_probs = model(bow_vector)
    print(log_probs)

Parameter containing:
tensor([[ 0.1011, -0.0866, -0.0380,  0.0921, -0.1846,  0.1176, -0.0403,  0.0998,
          0.0273, -0.0240,  0.0544,  0.0097,  0.0716, -0.0764, -0.0143, -0.0177,
          0.0284, -0.0008,  0.1714,  0.0610, -0.0730, -0.1184, -0.0329, -0.0846,
         -0.0628,  0.0094],
        [ 0.1169,  0.1066, -0.1917,  0.1216,  0.0548,  0.1860,  0.1294, -0.1787,
         -0.1865, -0.0946,  0.1722, -0.0327,  0.0839, -0.0911,  0.1924, -0.0830,
          0.1471,  0.0023, -0.1033,  0.1008, -0.1041,  0.0577, -0.0566, -0.0215,
         -0.1885, -0.0935]], requires_grad=True)
Parameter containing:
tensor([ 0.1064, -0.0477], requires_grad=True)
tensor([[-0.8195, -0.5810]])


In [7]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}


In [8]:
# 훈련하기 전에 테스트 데이터를 실행하여 전후를 볼 수 있습니다.
with torch.no_grad():
    for instance, label in test_data:
        bow_vec = make_bow_vector(instance, word_to_ix)
        log_probs = model(bow_vec)
        print(log_probs)

# "creo"에 해당하는 행렬 열을 인쇄하십시오.
print(next(model.parameters())[:, word_to_ix["creo"]])

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 일반적으로 교육 데이터를 여러 번 전달 합니다.
# 100은 실제 데이터 세트보다 훨씬 더 크지 만 실제 데이터 세트는 두 개 이상의 인스턴스를 가집니다.
# 일반적으로 5 ~ 30 개 에포크가 적당합니다.
for epoch in range(100):
    for instance, label in data:
        # 1 단계. PyTorch는 그라데이션을 축적합니다.
        # 각 인스턴스 전에 그들을 제거해야합니다.
        model.zero_grad()


        # 2 단계. BOW 벡터를 만들고 정수로 텐서로 대상을 싸야합니다.
        # 예를 들어, 대상이 SPANISH이면 정수 0으로 합니다.
        # 손실 함수는 로그 확률의 0번째 요소가 SPANISH에 해당하는 로그 확률임을 알 수 있습니다
        bow_vec = make_bow_vector(instance, word_to_ix)
        target = make_target(label, label_to_ix)

        # 3 단계. 순전파를 실행합니다.
        log_probs = model(bow_vec)

        # 4 단계. optimizer.step()을 호출하여 손실, 그래디언트를 계산하고 파라미터를 업데이트합니다.
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

with torch.no_grad():
    for instance, label in test_data:
        bow_vec = make_bow_vector(instance, word_to_ix)
        log_probs = model(bow_vec)
        print(log_probs)

# 스페인어에 해당하는 색인이 올라갑니다. 영어가 내려갑니다!!
print(next(model.parameters())[:, word_to_ix["creo"]])

tensor([[-0.6250, -0.7662]])
tensor([[-0.5870, -0.8119]])
tensor([0.0544, 0.1722], grad_fn=<SelectBackward>)
tensor([[-0.1210, -2.1721]])
tensor([[-2.7767, -0.0643]])
tensor([ 0.5004, -0.2738], grad_fn=<SelectBackward>)
